In [1]:
import pdbufr
import sys
import traceback
 
from eccodes import *
from math import isnan

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
import ipyleaflet
import ipywidgets as widgets

In [2]:
# Import bufr file of the TC track
df = pdbufr.read_bufr("../data/A_JSXX01ECEP250000_C_ECMP_20230725000000_tropical_cyclone_track_DOKSURI_124p6degE_17p6degN_bufr4.bin",
    columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
             "pressureReducedToMeanSeaLevel",))
# Save the number of ensembles in a list
members = df.ensembleMemberNumber.unique()

In [3]:
# Cyclone temporal horizon definition
# For DOKSURI forecast of the 25th July we have 41 snapshot with 6h step
start_dte = datetime(2023, 7, 25)
end_dte = datetime(2023, 8, 4)
step = timedelta(hours=6)
time_steps = pd.date_range(start_dte, end_dte, freq=step)

In [4]:
# Save all ensemble members track in a list called locations
locations = []
tracks_len = []
tracks_date = []
for m in members:
    dft = df[df.ensembleMemberNumber == m]
    dft_noNaN = dft.dropna()
    lat = dft_noNaN.latitude.values
    lon = dft_noNaN.longitude.values
    locs = [[lat[i], lon[i]] for i in range(len(lat))]
    track_len = len(locs)
    date_range = time_steps[:track_len]
    locations.append(locs)
    tracks_len.append(track_len)
    tracks_date.append(date_range)

In [5]:
## SELECT ENSEMBLE MEMBER TO PLOT ##
member = widgets.Dropdown(
    options = members.tolist(),
    value = 1,
    description = 'Ensemble:',
    diasbled=False,
)
member

Dropdown(description='Ensemble:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [6]:
## TEMPORAL HORIZON RANGE SLIDEBAR WITH DATES ##

options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in tracks_date[member.value - 1]]
index = (0, len(options)-1)

temporal_horizon_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    # description='Temporal Horizon:',
    description='Define horizon:',
    orientation='horizontal',
    layout={'width': '500px'},
    disabled=False,
)

# Increase the size of the descprition box of the slider and other styles
temporal_horizon_slider.style.description_width = '115px'
temporal_horizon_slider.style.handle_color = 'orange'

def print_date_range(date_range):
    start, end = date_range
    s = start.strftime('%d %b %Y %H:%M')
    e = end.strftime('%d %b %Y %H:%M')
    print(f'Temporal horizon: {s} - {e}')
    
# Link horizon slider to ensemble member selection
def update_horizon_ens(change):
    options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in tracks_date[change.new - 1]]
    temporal_horizon_slider.options = options
    temporal_horizon_slider.index = (0, len(options)-1)
member.observe(update_horizon_ens, names='value')

In [7]:
## TEMPORAL SLIDEBAR TO SELECT SNAPSHOT##
# Function to convert datetime to integer
def datetime_to_int(dt):
    return int((dt - start_dte).total_seconds() // step.total_seconds())

# Function to convert interger to datetime
def int_to_datetime(ts):
    return start_dte + ts * step

timestep_slider=widgets.IntSlider(
    min=0, 
    max=datetime_to_int(end_dte), 
    step=1, 
    value=0, 
    description='Timestep:')
    
def print_timestep(dt):
    date = int_to_datetime(dt)
    print('Selected date: ', date.strftime('%d %b %Y %H:%M'))
    
# Link the timestep selection slider to the horizon slider
def update_snapshot_slider(change):
    start, end = change.new
    timestep_slider.value = datetime_to_int(start.to_pydatetime())
    timestep_slider.min = datetime_to_int(start.to_pydatetime())
    timestep_slider.max = datetime_to_int(end.to_pydatetime())
    
temporal_horizon_slider.observe(update_snapshot_slider, names='value')

In [8]:
## INTERACTIVE MAP FOR TROPICAL CYCLONE TRACK ##
# Create the basemap for plotting
initial_lat_lon = (locations[member.value - 1][0][0], locations[member.value - 1][0][1])
tc_track_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.5,
)

# Add a custom zoom slider
# zoom_slider = widgets.IntSlider(description="Zoom level:", min=1, max=17, value=4)
# widgets.jslink((zoom_slider, "value"), (tc_track_map, "zoom"))
# widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position="topright")
# tc_track_map.add_control(widget_control1)

# Add tropical cyclone track to the map
track = ipyleaflet.Polyline(
    locations=locations[member.value - 1],
    color="blue" ,
    fill=False,
    name='Track',
)

# Link the cyclone track to the temporal horizon slider
def update_track_location(change):
    start, end = change.new
    start_index = datetime_to_int(start.to_pydatetime())
    end_index = datetime_to_int(end.to_pydatetime())
    track.locations = locations[member.value - 1][start_index:end_index+1]
temporal_horizon_slider.observe(update_track_location, names='value')

tc_track_map.add_layer(track)

# Add a marker to the map, for the snapshot selected
marker = ipyleaflet.CircleMarker(location=locations[member.value - 1][timestep_slider.value],
                                 draggable=False,
                                 name = 'Timestep',
                                 color = "orange",
                                 fill_color = "orange",
                                 radius = 2,
                                )

# Link the marker to the snapshot selections slider
def update_marker_location(change):
    marker.location = locations[member.value - 1][change.new]
timestep_slider.observe(update_marker_location, names='value')

tc_track_map.add_layer(marker)

# Add layers widget to the map
tc_track_map.add_control(ipyleaflet.LayersControl());

In [9]:
widgets.VBox([temporal_horizon_slider, timestep_slider])
widgets.interact(
        print_date_range,
        date_range=temporal_horizon_slider
    );
widgets.interact(
    print_timestep,
    dt=timestep_slider
);

tc_track_map

interactive(children=(SelectionRangeSlider(description='Define horizon:', index=(0, 27), layout=Layout(width='…

interactive(children=(IntSlider(value=0, description='Timestep:', max=40), Output()), _dom_classes=('widget-in…

Map(center=[17.7, 124.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…